In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

D:\Anaconda\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
path = 'archivos/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


In [4]:
dictionary = Dictionary(data['Pre-Processed'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]

# Train the topic model
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=50)

In [5]:
# Imprimir temas
model.print_topics(num_words=15)

[(0,
  '0.035*"bien" + 0.035*"si" + 0.021*"juego" + 0.020*"caja" + 0.019*"mal" + 0.018*"plata" + 0.017*"original" + 0.013*"switch" + 0.013*"mejor" + 0.013*"solo" + 0.012*"juegos" + 0.012*"viene" + 0.012*"nintendo" + 0.011*"precio" + 0.011*"bastante"'),
 (1,
  '0.037*"juego" + 0.035*"bonito" + 0.027*"bien" + 0.022*"cadena" + 0.020*"queda" + 0.019*"esperaba" + 0.017*"tal" + 0.017*"colgante" + 0.016*"pequeño" + 0.015*"parece" + 0.015*"foto" + 0.014*"aunque" + 0.013*"tamaño" + 0.012*"perfecto" + 0.012*"grande"'),
 (2,
  '0.045*"día" + 0.045*"llegado" + 0.042*"producto" + 0.026*"llegó" + 0.023*"si" + 0.022*"pulsera" + 0.022*"perfecto" + 0.015*"entrega" + 0.014*"entregado" + 0.013*"sido" + 0.013*"bien" + 0.013*"amazon" + 0.012*"pedido" + 0.012*"q" + 0.012*"descripción"'),
 (3,
  '0.081*"calidad" + 0.054*"buena" + 0.045*"precio" + 0.029*"mando" + 0.027*"buen" + 0.025*"si" + 0.022*"puede" + 0.016*"cierre" + 0.015*"diseño" + 0.015*"pedir" + 0.014*"hijo" + 0.013*"bueno" + 0.013*"producto" + 0.01

In [6]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)
# pyLDAvis.save_html(lda_display, 'lda.html')